In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from scipy.stats import zscore
from scipy import stats
from scipy.stats import kurtosis
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import warnings
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, r2_score
warnings.filterwarnings("ignore")



start = "2008-01-01"
end = "2025-01-01"


In [ ]:
index = yf.download("^GSPC", start =start, end= end)
vix = yf.download("^VIX", start=start, end=end)
skew = yf.download("^SKEW", start=start, end=end)



spx = index.add_prefix("SPX_")
vix   = vix.add_prefix("VIX_")
skew  = skew.add_prefix("SKEW_")


In [ ]:
df = spx.join(vix, how="inner").join(skew, how="inner")

if isinstance(df.columns, pd.MultiIndex):
    df.columns = [col[0] for col in df.columns]

df

In [ ]:
df["SPX_Return"] = np.log(df["SPX_Close"] / df["SPX_Close"].shift(1))  # Log returns
df["Abs_Return"] = df["SPX_Return"].abs()                               # Absolute returns
df["RealizedVol_21d"] = df["SPX_Return"].rolling(21).std() * np.sqrt(252)  # 21-day annualized vol
df["vol"] = df["SPX_Return"].rolling(5).std() * np.sqrt(252)            # 5-day annualized vol
df["VIX_Change"] = df["VIX_Close"].pct_change()                         # VIX daily change
df["IV_RV_Spread"] = (df["VIX_Close"] / 100) - df["RealizedVol_21d"]    # Implied - Realized vol
df["Vol_Ratio"] = (df["VIX_Close"] / 100) / df["RealizedVol_21d"]       # IV/RV ratio
df["SKEW_Change"] = df["SKEW_Close"].diff()                             # SKEW daily change

In [ ]:
df = df.dropna()  

df = df[['SPX_Close','SPX_Open',"SPX_High","SPX_Low",'SPX_Volume', 
        'VIX_Close','VIX_Open', 'SKEW_Close','SKEW_Open','SPX_Return', 'Abs_Return', 'RealizedVol_21d',
        'IV_RV_Spread', 'VIX_Change', 'SKEW_Change', 'Vol_Ratio','vol']]
df


In [ ]:
missing_value = df.isnull().sum()
print(missing_value)

In [ ]:
df["Daily_Range"] = (df["SPX_High"] - df["SPX_Low"]) / df["SPX_Close"]
df["VIX_Spike"] = (df["VIX_Close"] - df["VIX_Close"].rolling(5).mean()) / df["VIX_Close"].rolling(5).mean()
df["Vol_Acceleration"] = df["RealizedVol_21d"] - df["RealizedVol_21d"].rolling(5).mean()
df["Price_to_MA20"] = df["SPX_Close"] / df["SPX_Close"].rolling(20).mean() - 1
df["Return_Skew_10d"] = df["SPX_Return"].rolling(10).skew()
df["SKEW_Deviation"] = df["SKEW_Close"] - df["SKEW_Close"].rolling(10).mean()
df["Volume_Spike"] = df["SPX_Volume"] / df["SPX_Volume"].rolling(20).mean()
df

In [ ]:
# Interest Rates
df["US10Y"] = yf.download("^TNX", start=start, end=end)["Close"]
df["Yield_2Y"] = yf.download("^IRX", start=start, end=end)["Close"]
df["Yield_Curve"] = df["US10Y"] - df["Yield_2Y"]  # 10Y-2Y spread

# Dollar
df["DXY"] = yf.download("DX-Y.NYB", start=start, end=end)["Close"]
df["DXY_Change"] = df["DXY"].pct_change()

# Gold (safe haven)
df["Gold"] = yf.download("GC=F", start=start, end=end)["Close"]
df["Gold_Return"] = df["Gold"].pct_change()

# Credit Spreads
df["HYG"] = yf.download("HYG", start=start, end=end)["Close"]  # High Yield
df["LQD"] = yf.download("LQD", start=start, end=end)["Close"]  # Investment Grade
df["HYG_Return"] = df["HYG"].pct_change()
df["LQD_Return"] = df["LQD"].pct_change()
df["Credit_Spread_Returns"] = df["HYG_Return"] - df["LQD_Return"]  # Risky bonds - Safe bonds returns

df

In [ ]:
# Complete list of features 
df_final = df[[
        'SPX_Close',
        'SPX_Return',
        'RealizedVol_21d',
        'VIX_Close',
        'VIX_Change',
        'IV_RV_Spread',
        'Vol_Ratio',
        'Vol_Acceleration',
        'VIX_Spike',
        'vol',

        #SKEW
        'SKEW_Close',
        'SKEW_Change',
        'SKEW_Deviation',

        #Prix/Returns
        'Abs_Return',
        'Daily_Range',
        'Price_to_MA20',
        'Return_Skew_10d',

        #Volume
        'Volume_Spike',

        #Interest rate
        'Yield_Curve',  # 10Y-2Y spread (inversion = récession)

        #Devises
        'DXY_Change',  # Dollar Index

        #Commo
        'Gold_Return',

        # Additionnal features
        'HYG_Return',
        'LQD_Return',
        'Credit_Spread_Returns'

    ]]

df_final
df_final = df_final.dropna()

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error, r2_score

# parametres 
lookback = 30  # Utiliser 30 jours passés
forecast_horizon = 5  # Prédire 5 jours dans le futur
start_date = '2008-01-01'
split_date = '2019-12-31' # Séparation Train/Test avant la crise de la Covid-19

# reminder of our features of xgb
features1 = [
    'SPX_Return', 'VIX_Close', 'SKEW_Close', 'RealizedVol_21d',
    'IV_RV_Spread', 'Volume_Spike', 'Yield_Curve',
    'Gold_Return', 'Credit_Spread_Returns'
]

#definition of the target 
target_feature = 'RealizedVol_21d'


df_final['Future_Volatility'] = df_final[target_feature].shift(-forecast_horizon)
df_final.dropna(inplace=True)


X = df_final[features1].values
y = df_final['Future_Volatility'].values

#sequences creation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))

def create_sequences(data, labels, lookback):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i])
        y.append(labels[i])
    return np.array(X), np.array(y)

X_seq, y_seq = create_sequences(X_scaled, y_scaled.flatten(), lookback)

#train/test

#we try to find the split date, 2019 in the data, if not found then keyerror and we split 80% of the data for the train, lstm needs time series
try:
    split_idx_data = df_final.index.get_loc(split_date)
except KeyError:
    split_idx_data = int(len(df_final) * 0.8)

# index of separation must take into account the lookback
split_idx_seq = split_idx_data - lookback

if split_idx_seq < 0:
    split_idx_seq = int(len(X_seq) * 0.8)

X_train_lstm = X_seq[:split_idx_seq]
y_train_lstm = y_seq[:split_idx_seq]
X_test_lstm = X_seq[split_idx_seq:]
y_test_lstm = y_seq[split_idx_seq:]

#implementation of the model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(lookback, len(features1))),
    Dropout(0.2),
    LSTM(50, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

history = model.fit(
    X_train_lstm, y_train_lstm,
    epochs=20,
    batch_size=32,
    validation_data=(X_test_lstm, y_test_lstm),
    verbose=0 # Mis à 0 pour la clarté de la réponse
)

#prediction
y_pred_lstm = model.predict(X_test_lstm)
predictions = scaler.inverse_transform(y_pred_lstm)
y_test_real = scaler.inverse_transform(y_test_lstm.reshape(-1, 1))

#visualisation

dates_test_full = df_final.index[lookback:][split_idx_seq:]
dates_test = dates_test_full[:len(y_test_real)] # S'assurer que l'index a la même taille que y_test_real

# if the index is too short then the probleme is the initail indexation
if len(dates_test) != len(y_test_real):
    print(" Erreur d'alignement: Index de {len(dates_test)} dates vs {len(y_test_real)} points. Vérifiez split_date.")
    dates_test = pd.to_datetime(np.arange(len(y_test_real))) # default index
    
plt.figure(figsize=(14, 7))
plt.plot(dates_test, y_test_real, label=f"reel volatility ", color='blue', linewidth=2)
plt.plot(dates_test, predictions, label=f"lstm predictions (5 days)", color='red', linestyle='dashed', linewidth=1.5)

plt.axvspan(pd.to_datetime('2020-02-20'), pd.to_datetime('2020-04-01'), color='gray', alpha=0.3, label='Crise COVID-19')

plt.title(f"volatility prediction using lstm", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Annualized volatility", fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.5)
plt.show()